# Acesso a bases de dados MySQL com Python  
Pedro Cardoso

(ISE/UAlg - pcardoso@ualg.pt)

## Estabelecimento de conexão à base de dados usando um Connector/Python

o método  `connect()` cria uma conexão a um servidor MySQL e devolve um objeto `MySQLConnection`.
(ver https://dev.mysql.com/doc/connector-python/en/connector-python-connectargs.html para outros argumentos e opções)

In [2]:
import mysql.connector

cnx = mysql.connector.connect(user='adam', 
                              password='adam',
                              host='localhost', # replace 'localhost', if necessary
                              database='adamastor')
cnx

DatabaseError: 2003 (HY000): Can't connect to MySQL server on 'localhost:3306' (10061)

Algumas informações sobre a conexão podem ser consultadas no `__dict__`

In [4]:
cnx.__dict__

{'_cmysql': <_mysql_connector.MySQL at 0x2264a00>,
 '_columns': [],
 '_plugin_dir': '/home/pcardoso/.local/lib/python3.8/site-packages/mysql/vendor',
 'converter': None,
 '_client_flags': 1286669,
 '_charset_id': 45,
 '_sql_mode': None,
 '_time_zone': None,
 '_autocommit': False,
 '_server_version': (5, 5, 5),
 '_handshake': {'protocol': 10,
  'server_version_original': '5.5.5-10.4.11-MariaDB',
  'server_threadid': 20,
  'charset': None,
  'server_status': None,
  'auth_plugin': None,
  'auth_data': None,
  'capabilities': -2113931266},
 '_conn_attrs': {'_connector_name': 'mysql-connector-python',
  '_connector_license': 'GPL-2.0',
  '_connector_version': '8.0.23',
  '_source_host': 'pcardoso-kubuntu-desktop'},
 '_user': 'adam',
 '_password': 'adam',
 '_database': 'adamastor',
 '_host': 'localhost',
 '_port': 3306,
 '_unix_socket': None,
 '_client_host': '',
 '_client_port': 0,
 '_ssl': {},
 '_ssl_disabled': False,
 '_force_ipv6': False,
 '_use_unicode': True,
 '_get_warnings': False,


E no final devemos libertar sempre a conexão

In [5]:
cnx.close()

### Ficheiro de configuração  e tratamento de exceções
De um modo geral é aconselhável
* fazer tratamento de exceções 
* e criar um ficheiro de configuração (config.py) 
```
config = {
    'host' : 'localhost',
    'user' : 'adam',
    'password' : 'adam',
    'db' : 'adamastor'
}
```
e depois fazer...

In [7]:
# Comecamos por importar o ficheiro de configuração
import mysql.connector

from config import config
config

{'host': 'localhost', 'user': 'adam', 'password': 'adam', 'db': 'adamastor'}

In [8]:
try:
    cnx = mysql.connector.connect(**config)
except mysql.connector.Error as err:
    print('Ups! Ocorreu um erro!')
    print(dir(err))
    print(err.errno)
else:
    print('Sucesso!')
    cnx.close()

Sucesso!


### Exercício
1. Experimentem a desligar o servidor e correr a linha acima: qual a mensagem de erro?
2. Mude o nome do utilizador no ficheiro de configuração (reinicie o kernel) e corra a linha acima: qual a mensagem de erro?
3. Re-implemente o código de modo a dar a mensagem adequada quando falhar pelo servidor estar desligado e voltar a tentar mais 2 vezes a cada 5 segundos (vejam o pacote `time` e particular o método `sleep()`)

In [9]:
import time
print('ola')
time.sleep(5)
print('ola de novo!')

ola
ola de novo!


## Operações de DDL: Criação de uma base de dados

Para a criação das tabelas e relacionamentos podemos construiro o sql ou, como alternativa, podemos usar ferramentas como sejam o MySQL Workbench, o Phpmyadmin, o SQlite Browser, o DataGrip, etc. 


Consideremos o caso em que a base de dados já existe...

## Operações CRUD

### `INSERT` 

Aberta a conexão em MySQL, O comando `cnx.cursor()` devolve um objeto da classe `MySQLCursor` que podem executar operações como instruções SQL. Objetos de `cursor` interagem com o servidor MySQL usando um objeto `MySQLConnection`. Para mais informações ver https://dev.mysql.com/doc/connector-python/en/connector-python-api-mysqlcursor.html

In [11]:
cnx = mysql.connector.connect(**config)
cursor = cnx.cursor()
cursor

#### localização
Uma boa estratégia é definir variáveis no SQL usando parametros no estilo `%s` or `%(nome)s` (i.e., usar o estilo "format" ou "pyformat" - ver https://pyformat.info/) e um tuplo com os dados

In [14]:
sql = '''
    INSERT INTO categorias
        (CódigoDaCategoria, NomeDaCategoria, Descrição, Imagem) 
    VALUES 
        (DEFAULT, %s, %s, NULL)
'''
# e um tuplo com os dados
data = ("Teste-Nome", "Teste-descrição")

e agora inserir uma nova categoria na base de dados e obter o id correspondente, guardado em `cat_id` e que poderemos usar à frente

In [15]:
cursor.execute(sql, data)
categoria_id = cursor.lastrowid
categoria_id_id

9

Importante, quando estamos a usar um sistema transacional, como o InnoDB, temos de efetuar o "commit" depois de fazer um INSERT, DELETE, ou UPDATE (comandos que alterem tabelas).
ver (https://dev.mysql.com/doc/connector-python/en/connector-python-api-mysqlconnection-commit.html)

In [16]:
cnx.commit()

#### Transitários
Inserir um nova `transitários`, se não existir (ver a documentação do comando `REPLACE` do SQL)

Preparar os dados, agora com um dicionário que tem em conta `%(nome)s, %(telefone)s`


In [18]:
sql = ''' 
    REPLACE INTO transitarios
        (CódigoDoTransitário, NomeDaEmpresa, Telefone)
    VALUES
        (DEFAULT, %(nome)s, %(telefone)s)
'''

data = {
    "nome": "UPS", 
    "telefone": "(351)289800800"
}

cursor.execute(sql, data)
cnx.commit()

In [20]:

transitatio_id = cursor.lastrowid
transitatio_id

5

In [21]:
cnx.close()

## Selecionar dados

Todo o processo é simples dados os conhecimentos anteriores

In [22]:
cnx = mysql.connector.connect(**config)
cursor = cnx.cursor()  

In [27]:
sql = '''
    SELECT CódigoDoTransitário, NomeDaEmpresa, Telefone
    FROM transitarios;
'''

cursor.execute(sql)

In [28]:
cursor

E percorrer os dados, usando por exemplo um ciclo `for`

In [29]:
for (codigo, nome, telefone) in cursor:
    print("id: {}\n\t name: {} \n\t description: {}".format(codigo, nome, telefone))


id: 1
	 name: Speedy Express 
	 description: (503) 555-9831
id: 2
	 name: United Package 
	 description: (503) 555-3199
id: 3
	 name: Federal Shipping 
	 description: (503) 555-9931
id: 5
	 name: UPS 
	 description: (351)289800800


Outros exemplos

In [45]:
sql = '''
SELECT CódigoDoCliente,
    NomeDaEmpresa,
    Cidade,
    País,
    Telefone
FROM clientes
WHERE País BETWEEN %s AND %s
ORDER BY País
'''
data = ("N", "Q")

cursor.execute(sql, data)

for (CódigoDoCliente, NomeDaEmpresa, Cidade, País, Telefone) in cursor:
    print(f"""CódigoDoCliente: {CódigoDoCliente}\n\t NomeDaEmpresa: {NomeDaEmpresa} \n\t Cidade: {Cidade} \n\t País: {País} \n\t Telefone: {Telefone}""")

CódigoDoCliente: CódigoDoCliente
	 NomeDaEmpresa: NomeDaEmpresa 
	 Cidade: Cidade 
	 País: País 
	 Telefone: Telefone
CódigoDoCliente: CódigoDoCliente
	 NomeDaEmpresa: NomeDaEmpresa 
	 Cidade: Cidade 
	 País: País 
	 Telefone: Telefone
CódigoDoCliente: CódigoDoCliente
	 NomeDaEmpresa: NomeDaEmpresa 
	 Cidade: Cidade 
	 País: País 
	 Telefone: Telefone
CódigoDoCliente: CódigoDoCliente
	 NomeDaEmpresa: NomeDaEmpresa 
	 Cidade: Cidade 
	 País: País 
	 Telefone: Telefone


Podemos obter os nomes e outros dados das colunas (https://dev.mysql.com/doc/connector-python/en/connector-python-api-mysqlcursor-description.html) no formato
(name, type_code, display_size, internal_size, precision, scale, null_ok, column_flags)


In [34]:
cursor.description

[('CódigoDoCliente', 253, None, None, None, None, 1, 16388),
 ('NomeDaEmpresa', 253, None, None, None, None, 1, 16392),
 ('Cidade', 253, None, None, None, None, 1, 16392),
 ('País', 253, None, None, None, None, 1, 0),
 ('Telefone', 253, None, None, None, None, 1, 0)]

In [35]:
from mysql.connector import FieldType

for i in range(len(cursor.description)):
    print("Column {}:".format(i+1))
    desc = cursor.description[i]
    print("  column_name = {}".format(desc[0]))
    print("  type = {} ({})".format(desc[1], FieldType.get_info(desc[1])))
    print("  null_ok = {}".format(desc[6]))

Column 1:
  column_name = CódigoDoCliente
  type = 253 (VAR_STRING)
  null_ok = 1
Column 2:
  column_name = NomeDaEmpresa
  type = 253 (VAR_STRING)
  null_ok = 1
Column 3:
  column_name = Cidade
  type = 253 (VAR_STRING)
  null_ok = 1
Column 4:
  column_name = País
  type = 253 (VAR_STRING)
  null_ok = 1
Column 5:
  column_name = Telefone
  type = 253 (VAR_STRING)
  null_ok = 1


In [36]:
lista_de_colunas = [linha[0] for linha in cursor.description]
lista_de_colunas

['CódigoDoCliente', 'NomeDaEmpresa', 'Cidade', 'País', 'Telefone']

### o comando `fetchall`
Usando o comando `fetchall` podemos obter todos os resultados de uma única vez como uma lista de tuplos

In [39]:
# é necessario voltar a correr o select pois o cursor foi esvaziado
cursor.execute(sql, data)

cursor.fetchall()

[('SANTG', 'Santé Gourmet', 'Stavern', 'Noruega', '07-98 92 35'),
 ('WOLZA', 'Wolski  Zajazd', 'Warszawa', 'Polónia', '(26) 642-7012'),
 ('FURIB',
  'Furia Bacalhau e Frutos do Mar',
  'Lisboa',
  'Portugal',
  '(1) 354-2534'),
 ('PRINI', 'Princesa Isabel Vinhos', 'Lisboa', 'Portugal', '(1) 356-5634')]

Podemos também converter para um dicionário

In [41]:
# é necessario voltar a correr o select pois o cursor foi "esvaziado"
cursor.execute(sql, data)

for linha in cursor:
    print({coluna: valor for valor, coluna  in zip(linha, lista_de_colunas)})

{'CódigoDoCliente': 'SANTG', 'NomeDaEmpresa': 'Santé Gourmet', 'Cidade': 'Stavern', 'País': 'Noruega', 'Telefone': '07-98 92 35'}
{'CódigoDoCliente': 'WOLZA', 'NomeDaEmpresa': 'Wolski  Zajazd', 'Cidade': 'Warszawa', 'País': 'Polónia', 'Telefone': '(26) 642-7012'}
{'CódigoDoCliente': 'FURIB', 'NomeDaEmpresa': 'Furia Bacalhau e Frutos do Mar', 'Cidade': 'Lisboa', 'País': 'Portugal', 'Telefone': '(1) 354-2534'}
{'CódigoDoCliente': 'PRINI', 'NomeDaEmpresa': 'Princesa Isabel Vinhos', 'Cidade': 'Lisboa', 'País': 'Portugal', 'Telefone': '(1) 356-5634'}


In [42]:
cursor.close()
cnx.close()

### Dados na forma de dicionários

Se criar o cursor com o parametro `dictionary=True` ao iterar sobre os resultados estes vêm na forma de dicionários

In [49]:
cnx = mysql.connector.connect(**config)
cursor = cnx.cursor(dictionary=True)

cursor.execute(sql, data)

from pprint import pprint
pprint(cursor.fetchall())

[{'Cidade': 'Stavern',
  'CódigoDoCliente': 'SANTG',
  'NomeDaEmpresa': 'Santé Gourmet',
  'País': 'Noruega',
  'Telefone': '07-98 92 35'},
 {'Cidade': 'Warszawa',
  'CódigoDoCliente': 'WOLZA',
  'NomeDaEmpresa': 'Wolski  Zajazd',
  'País': 'Polónia',
  'Telefone': '(26) 642-7012'},
 {'Cidade': 'Lisboa',
  'CódigoDoCliente': 'FURIB',
  'NomeDaEmpresa': 'Furia Bacalhau e Frutos do Mar',
  'País': 'Portugal',
  'Telefone': '(1) 354-2534'},
 {'Cidade': 'Lisboa',
  'CódigoDoCliente': 'PRINI',
  'NomeDaEmpresa': 'Princesa Isabel Vinhos',
  'País': 'Portugal',
  'Telefone': '(1) 356-5634'}]


In [50]:
cursor.close()
cnx.close()